In [56]:
import altair as alt
import pandas as pd
import sqlalchemy

In [10]:
engine = sqlalchemy.engine.create_engine("sqlite:///municipal_performance.sqlite3")

In [85]:
query = """
SELECT municipalities.name_fr as mun, e.year, e.score, c.max_score, c.name_fr
FROM municipalities
         JOIN evaluations e ON municipalities.municipality_id = e.municipality_id
         JOIN criteria c ON e.criterion_id = c.criterion_id
WHERE c.parent_id NOT IN (SELECT c.criterion_id WHERE c.parent_id IS NULL)
  AND municipalities.municipality_id = 1313
ORDER BY municipalities.name_fr, e.year, c.max_score DESC;
"""
df = pd.read_sql(query, engine)

In [86]:
df.head()

,mun,year,score,max_score,name_fr
0,Hammam Lif,2017,15.0,15,Entretiens des Biens
1,Hammam Lif,2017,8.0,12,Approche Participative
2,Hammam Lif,2017,5.0,10,Mécanisme de gestion des plaintes
3,Hammam Lif,2017,10.0,10,Ressources Humaines
4,Hammam Lif,2017,1.0,10,Gestion Financière


In [123]:
df.melt(id_vars=["year", "name_fr"], value_vars=["score", "max_score"])

,year,name_fr,variable,value
0,2017,Entretiens des Biens,score,15.0
1,2017,Approche Participative,score,8.0
2,2017,Mécanisme de gestion des plaintes,score,5.0
3,2017,Ressources Humaines,score,10.0
4,2017,Gestion Financière,score,1.0
...,...,...,...,...
139,2018,Point focal pour la gestion des plaintes nommé...,max_score,2.0
140,2018,Un tableau de bord sur le progrès des commande...,max_score,2.0
141,2018,Les délais de clôture définitive des contrats ...,max_score,2.0
142,2018,Mise à jour annuel du rôle de recouvrement des...,max_score,2.0


In [141]:
df.melt(id_vars=["year", "name_fr"], value_vars=["score", "max_score"]).pipe(
    lambda df: alt.Chart(df)
    .mark_bar()
    .encode(
        y="value:Q",
        x="variable:N",
        column="name_fr",
        color="variable:N",
        row="year:O",
        tooltip=["name_fr:N", "variable:N", "value:Q"],
    )
    .properties(width=50)
)

alt.Chart(...)